# Import 

!pip install pandas openai transformers

In [2]:
import pandas as pd
from openai import OpenAI

# DATA

In [17]:
df = pd.read_csv("raw_queries.csv")
df.head()
df["source"].value_counts()

source
ms_marco             20000
natural_questions    19999
rag_12000             9590
Name: count, dtype: int64

# Models

In [16]:
from openai import OpenAI

# Initialise le client Groq (à faire une seule fois)
client = OpenAI(
    api_key="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",
    base_url="https://api.groq.com/openai/v1"
)

# Prompt template avec {query}
PROMPT_TEMPLATE = """
Your task is to take a raw search query and rewrite it to improve retrieval performance in a search engine or RAG system.

Follow these guidelines:
- Correct any grammar or spelling errors.
- Rephrase the query naturally and clearly.
-  Add synonyms or related terms only to the most informative words (rare, domain-specific) in parentheses
- Keep it concise and relevant.

DO NOT SAY ANYTHING ELSE THAN THE QUERY

### Example
Input: danger screen sleep 
Output: What are the risks (hazards, dangers) of using screens (monitors, displays) before sleep (slumber, bedtime)?

Now rewrite this query:
Input: {query}
"""

def rewrite_query_llm(query: str, temperature: float = 0.5) -> str:
    prompt = PROMPT_TEMPLATE.format(query=query)

    try:
        response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=128
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"[ERROR] {e}"

In [7]:
from transformers import pipeline
import nltk
import spacy
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('punkt')
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

# 1. Correction — modèle bien configuré
corrector = pipeline("text2text-generation", model="vennify/t5-base-grammar-correction")

# 2. Reformulation
paraphraser = pipeline("text2text-generation", model="Vamsi/T5_Paraphrase_Paws")

# 3. Expansion via WordNet sur les mots clés
def synonymize_key_terms(text, max_synonyms=2):
    doc = nlp(text)
    final_tokens = []
    for token in doc:
        if token.pos_ in {"NOUN", "VERB", "ADJ"} and token.is_alpha and len(token.text) > 3:
            syns = wordnet.synsets(token.text)
            lemmas = list(set(
                lemma.name().replace('_', ' ') 
                for s in syns for lemma in s.lemmas() 
                if lemma.name().lower() != token.text.lower()
            ))
            if lemmas:
                final_tokens.append(f"{token.text} ({', '.join(lemmas[:max_synonyms])})")
            else:
                final_tokens.append(token.text)
        else:
            final_tokens.append(token.text)
    return " ".join(final_tokens)

# Pipeline complet
def rewrite_pipeline(query, verbose=False):
    corrected = corrector(query, max_length=64, do_sample=False)[0]["generated_text"]
    explicit = paraphraser(corrected, max_length=64, do_sample=False)[0]["generated_text"]
    enriched = synonymize_key_terms(explicit)

    if verbose:
        print(f"\nRaw        : {query}")
        print(f"Corrected  : {corrected}")
        print(f"Explicit   : {explicit}")
        print(f"Enriched   : {enriched}")
    return enriched


/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 93.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Device set to use cpu
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cpu


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Chargement du modèle Flan-T5
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


FLAN_PROMPT_TEMPLATE = """
Rewrite the following query to improve information retrieval.
- Correct grammar or spelling
- Rephrase it naturally and clearly
- Add synonyms or related terms only to the most informative words (rare, domain-specific) in parentheses
- Keep it short and useful

### Example
Input: danger screen sleep
Output: What are the risks (hazards, dangers) of using screens (monitors, displays) before sleep (slumber, bedtime)?

Now rewrite this query:
Input: {query}
"""

# Création du pipeline
flan_pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Fonction de rewriting
def rewrite_with_flan(query: str, temperature=0.5, max_tokens=100) -> str:
    prompt = FLAN_PROMPT_TEMPLATE.format(query=query)
    result = flan_pipe(
        prompt,
        max_length=max_tokens,
        do_sample=True,
        temperature=temperature
    )[0]["generated_text"]
    return result.strip()

Device set to use cpu


# Tests comparatifs

In [17]:
sample = df.sample(30, random_state=123)

In [19]:
sample_queries_list = sample['query'].values
print(sample_queries_list)

['what do chromosomes look like in bacteria'
 'what are chromosomes kid definition'
 'who coined the term aerobics and in what year'
 "What is the proposed change to the parental leave payments scheme by the Coalition's small business spokesman, Mr Billson?"
 "What is Sean Kane's educational background?"
 "What is the concern of the Savannah Tree Foundation regarding the new law school's parking lot?"
 'name the animal from which we get wool' 'what is keg coupler'
 'What are the functions of the KitchenAid Mixer attachment pack?'
 'where is uc santa barbara located'
 'what is mitochondrial disease symptoms' 'liable definition law'
 'what language do people from iceland speak'
 'What is the author struggling with?'
 'wii fit u vs wii fit plus difference'
 'who sang if you like pina coladas lyrics'
 'who are the freedom riders and what are they trying to accomplish'
 'best method to cook a sirloin tip steak'
 'What is the legal status of prostitution in Hamilton, Canada?'
 'What is the p

In [11]:
%%time
for q in sample_queries_list:
    print(q)
    print(FLAN_PROMPT_TEMPLATE.format(query=q))
    print(rewrite_with_flan(q))
    print('-'*50)

yahya name meaning

Rewrite the following query to improve information retrieval.
- Correct grammar or spelling
- Rephrase it naturally and clearly
- Add synonyms or related terms only to the most informative words (rare, domain-specific) in parentheses
- Keep it short and useful

### Example
Input: danger screen sleep
Output: What are the risks (hazards, dangers) of using screens (monitors, displays) before sleep (slumber, bedtime)?

Now rewrite this query:
Input: yahya name meaning



Output: The meaning of the name yahya is:
--------------------------------------------------
where does the electron transport chain occur in prokaryotes and eukaryotes

Rewrite the following query to improve information retrieval.
- Correct grammar or spelling
- Rephrase it naturally and clearly
- Add synonyms or related terms only to the most informative words (rare, domain-specific) in parentheses
- Keep it short and useful

### Example
Input: danger screen sleep
Output: What are the risks (hazards, dangers) of using screens (monitors, displays) before sleep (slumber, bedtime)?

Now rewrite this query:
Input: where does the electron transport chain occur in prokaryotes and eukaryotes

Where does the electron transport chain occur in prokaryotes and eukaryotes?
--------------------------------------------------
what happened to the house in amityville horror

Rewrite the following query to improve information retrieval.
- Correct grammar or spelling
- Rephrase it naturally and clearl

In [20]:
%%time
for q in sample_queries_list[18:20]:
    print("-"*50)
    print(rewrite_query_llm(q,))

--------------------------------------------------
What is the legal framework (regulations, laws) surrounding prostitution in Hamilton, Ontario, Canada?
--------------------------------------------------
What is the objective (goal, aim) of the 2015 report compiled by the Pakistan Federal Union of Journalists (PFUJ)?
CPU times: user 28.2 ms, sys: 12.4 ms, total: 40.6 ms
Wall time: 703 ms


In [13]:
df.shape

(52823, 2)

# Tests on refined_queries

In [69]:
refined_queries_df = pd.read_csv("refined_queries.csv")
df = pd.read_csv("raw_queries.csv")

In [70]:
df["query"] = df["query"].str.strip()
refined_queries_df['query'] = refined_queries_df['query'].str.strip()

In [72]:
paired_queries_df = pd.merge(refined_queries_df, df, on="query", how="left")

# Tests on paired_queries

In [ ]:
paired_queries_df = pd.read_csv("paired_queries.csv")


In [79]:
paired_queries_df.shape

(9973, 4)

In [80]:
paired_queries_df_train, paired_queries_df_test = paired_queries_df.loc[:9800, :], paired_queries_df.loc[9800:, :]

In [85]:
paired_queries_df_train.to_csv("paired_queries_train.csv", index=False)

In [84]:
paired_queries_df_test.to_csv("paired_queries_test.csv", index=False)

# Training

In [ ]:
import subprocess
subprocess.run(["pip", "install", "transformers", "scikit-learn", "openai", "accelerate", "pandas", "wandb", "datasets", "peft"])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 31.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 34.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.4/644.4 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 49.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 28.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 34.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2025.3.2 requires fsspec==2025.3.2.*, but you have fsspec 2024.12.0 which is incompatible.


CompletedProcess(args=['pip', 'install', 'transformers', 'scikit-learn', 'openai', 'accelerate', 'pandas', 'wandb', 'datasets'], returncode=0)

In [2]:
subprocess.run(["wandb", "login", "d1beea7e0d2ac446ba4460daa9aff7ddfc3fe41c"])

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /home/onyxia/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


CompletedProcess(args=['wandb', 'login', 'd1beea7e0d2ac446ba4460daa9aff7ddfc3fe41c'], returncode=0)

In [17]:
import subprocess
subprocess.run(["python", "finetunning_T5_LORA.py"])

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vincent-gimenes to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /home/onyxia/work/NLP_ENSAE/wandb/run-20250413_150603-zt1xkc7b
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run flan-t5-large-5epochs-lora
wandb: ⭐️ View project at https://wandb.ai/NLP_ENSAE/finetunning%20T5
wandb: 🚀 View run at https://wandb.ai/NLP_ENSAE/finetunning%20T5/runs/zt1xkc7b


trainable params: 9,437,184 || all params: 792,587,264 || trainable%: 1.1907


Map: 100%|██████████| 491/491 [00:00<00:00, 1489.14 examples/s]
/home/onyxia/work/NLP_ENSAE/finetunning_T5_LORA.py:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
  2%|▏         | 50/2910 [00:41<41:31,  1.15it/s] 

{'loss': 2.609, 'grad_norm': 0.5104425549507141, 'learning_rate': 8.90909090909091e-05, 'epoch': 0.09}


  3%|▎         | 100/2910 [01:24<38:06,  1.23it/s]

...

{'loss': 1.1031, 'grad_norm': 0.686771810054779, 'learning_rate': 2.0872865275142314e-06, 'epoch': 4.98}


100%|██████████| 2910/2910 [42:28<00:00,  1.14it/s]


{'train_runtime': 2548.2639, 'train_samples_per_second': 18.267, 'train_steps_per_second': 1.142, 'train_loss': 1.2553657659550304, 'epoch': 5.0}


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:               eval/loss █▅▄▃▃▂▂▂▂▁▁▁▁▁
wandb:            eval/runtime █▃▂▁▁▁▁▁▁▁▂▁▂▂
wandb: eval/samples_per_second ▁▆▇███████▇█▇▇
wandb:   eval/steps_per_second ▁▆▇███▇███▇█▇▇
wandb:             train/epoch ▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇██
wandb:       train/global_step ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:         train/grad_norm ▃▃▃▂▂▁▂▃▁▂▁▃▃▂▅▄▄▄▂▄▇▄▄▃▅▇█▄▄▃▆▅▃▆▄▆▇▅▆█
wandb:     train/learning_rate ▂▄▆▇███▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
wandb:              train/loss █▃▃▃▂▂▂▂▂▂▂▂���▂▂▂▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:                eval/loss 1.04101
wandb:             eval/runtime 12.0991
wandb:  eval/samples_per_second 40.581
wandb:    eval/steps_per_second 2.562
wandb:               total_flos 6832462296563712.0
wandb:              train/epoch 5
wandb:        train/global_step 2910
wandb:          train/grad_norm 

CompletedProcess(args=['python', 'finetunning_T5_LORA.py'], returncode=0)

# Tests post training

In [13]:
import pandas as pd
df_test = pd.read_csv('data_folder/paired_queries_test.csv')

In [14]:
queries = df_test["noisy_query"].values[10:20]

## T5 Small

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Chargement du modèle fine-tuné
model_path = "./flan-t5-small-rewriting"  # ou base/large selon ce que tu as entraîné
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")

In [11]:
for query in queries:
    input_text = "Improve the query: " + query

    # Tokenisation
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    # Génération
    outputs = model.generate(**inputs, max_new_tokens=64)
    rewritten = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("🔎 Input query :", query)
    print("✍️ Rewritten   :", rewritten)

🔎 Input query : ho vouces dale on kinf of the hill
✍️ Rewritten   : What is the significance (importance, significance) of the dale on the kinf of the hill (sea, sand)?
🔎 Input query : ha features ewre availalt on B-R for th e0211 ppeing Dauz
✍️ Rewritten   : What features (features, features) are available on B-R for the e0211 season (season, season)?
🔎 Input query : wha is the color of horseshoe crab blood
✍️ Rewritten   : What is the color (color, color) of horseshoe crab blood (blood, blood)?
🔎 Input query : did king david ever existed
✍️ Rewritten   : Did King David ever exist (exist, exist) in the United States (USA, America)?
🔎 Input query : What is the primary purpose of any corporation according to economist Roger Martin?
✍️ Rewritten   : What is the primary objective (goal, objective) of any corporation according to economist Roger Martin?
🔎 Input query : what is teacjing english as a second language
✍️ Rewritten   : What is the primary language (language, language) of Englis

## T5 base

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Chargement du modèle fine-tuné
model_path = "./flan-t5-base-rewriting"  
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")

In [13]:
for query in queries:
    input_text = "Improve the query: " + query

    # Tokenisation
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    # Génération
    outputs = model.generate(**inputs, max_new_tokens=64)
    rewritten = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("🔎 Input query :", query)
    print("✍️ Rewritten   :", rewritten)

🔎 Input query : ho vouces dale on kinf of the hill
✍️ Rewritten   : Who plays the role of Dale on the TV show "The Kinf of the Hill"?
🔎 Input query : ha features ewre availalt on B-R for th e0211 ppeing Dauz
✍️ Rewritten   : What features are available on B-R for the 0211 season of the TV show "The Dauz"?
🔎 Input query : wha is the color of horseshoe crab blood
✍️ Rewritten   : What is the color (color, hue) of horseshoe crab blood (blood, blood)?
🔎 Input query : did king david ever existed
✍️ Rewritten   : Did King David ever exist (exist, exist)?
🔎 Input query : What is the primary purpose of any corporation according to economist Roger Martin?
✍️ Rewritten   : What is the primary objective (goal, objective) of any corporation according to economist Roger Martin's perspective (viewpoint, perspective)?
🔎 Input query : what is teacjing english as a second language
✍️ Rewritten   : What is the significance (importance, significance) of English as a second language (language, dialect)?
🔎

## T5 Large (lora 1)

In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Chargement du modèle fine-tuné
model_path = "./flan-t5-large-rewriting-lora"  
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
for query in queries:
    input_text = "Improve the query: " + query

    # Tokenisation
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    # Génération
    outputs = model.generate(**inputs, max_new_tokens=64)
    rewritten = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("🔎 Input query :", query)
    print("✍️ Rewritten   :", rewritten)

🔎 Input query : ho vouces dale on kinf of the hill
✍️ Rewritten   : What is the name of the valley (dale, valley) on the summit of Mount Kilimanjaro?
🔎 Input query : ha features ewre availalt on B-R for th e0211 ppeing Dauz
✍️ Rewritten   : What are the features (features, amenities) available on B-R for the E0211 eing Dauz?
🔎 Input query : wha is the color of horseshoe crab blood
✍️ Rewritten   : What is the color (tone, hue) of horseshoe crab blood (blood, blood)?
🔎 Input query : did king david ever existed
✍️ Rewritten   : Did King David (the King of Israel) ever exist (exist, exist)?
🔎 Input query : What is the primary purpose of any corporation according to economist Roger Martin?
✍️ Rewritten   : What is the primary objective (purpose, goal) of any corporation (organization, business) according to economist Roger Martin?
🔎 Input query : what is teacjing english as a second language
✍️ Rewritten   : What is the definition (meaning, meaning) of teaching English as a second language

## T5 Large (lora 2)

In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Chargement du modèle fine-tuné
model_path = "./flan-t5-large-rewriting-lora-2"  
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")

In [20]:
for query in queries:
    input_text = "Improve the query: " + query

    # Tokenisation
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    # Génération
    outputs = model.generate(**inputs, max_new_tokens=64)
    rewritten = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("🔎 Input query :", query)
    print("✍️ Rewritten   :", rewritten)

🔎 Input query : ho vouces dale on kinf of the hill
✍️ Rewritten   : Who plays the role of Dale (character, character) on the TV show "Kinf of the Hill"?
🔎 Input query : ha features ewre availalt on B-R for th e0211 ppeing Dauz
✍️ Rewritten   : What features are available on B-River (B-River Radio) for the February 11th episode of the TV show Dauz?
🔎 Input query : wha is the color of horseshoe crab blood
✍️ Rewritten   : What is the color of horseshoe crab blood (blood, fluid)?
🔎 Input query : did king david ever existed
✍️ Rewritten   : Did King David ever exist (exist, exist) in history (history, history)?
🔎 Input query : What is the primary purpose of any corporation according to economist Roger Martin?
✍️ Rewritten   : What is the primary objective (goal, aim) of any business (corporation, firm) according to economist Roger Martin's perspective (viewpoint, perspective)?
🔎 Input query : what is teacjing english as a second language
✍️ Rewritten   : What is the process (method, techni